### Imports

In [1]:
# Standard Imports

# Azure SDK Imports
from azure.ai.ml import Input, MLClient, Output, command
from azure.ai.ml.dsl import pipeline
from azure.identity import DefaultAzureCredential

# Third-party Imports

# Local Imports

### Initialize ML Client

In [2]:
ml_client = MLClient(
    credential=DefaultAzureCredential(),
    subscription_id="<your_subscription_id>",  # example: os.getenv("AML_SUBSCRIPTION_ID"),
    resource_group_name="<your_resource_group_name>",
    workspace_name="<your_workspace_name>",
)

In [3]:
ml_client.from_config(credential=DefaultAzureCredential())

Found the config file in: /home/daniel/repos/aml/config.json


MLClient(credential=<azure.identity._credentials.default.DefaultAzureCredential object at 0x7f241c339210>,
         subscription_id=11f51dee-57cd-4d47-b542-8e244706e163,
         resource_group_name=sbx-dondorp,
         workspace_name=amlsbxdondorp)

### Define Environment

In [ ]:
environment_name = "<your_environment_name>"
environment_version = "<your_environment_version>"

### Configure Job Component

In [13]:
job_component = command(
    code="./",
    command="python <your_script_name>.py",
    name="<your_job_name>",
    display_name="<your_job_display_name>",  # Name for identification in the portal
    compute="<your_compute_target>",  # In our case the name of our compute cluster
    environment=f"azureml:{environment_name}:{environment_version}",
    inputs={
        # Model Example
        "example_model": Input(type="custom_model", path="<registered_model_asset_id>"),
        # Folder Example
        "images": Input(
            type="uri_folder",
            path="azureml://subscriptions/<subscription_id>/resourcegroups/<your_resource_group>/workspaces/<your_workspace>/datastores/<your_datastore>/paths/<path_to_image_directory>/",
        ),
    },
    outputs={"output_dir": Output(type="uri_folder", path="azureml://<output-path>/")},
)

### Define Pipeline

In [14]:
@pipeline()
def run_pipeline():
    outputs = job_component()
    return outputs

### Submit Pipeline Job

In [ ]:
experiment_name = "<your_experiment_name>"
pipeline_job = run_pipeline()
job = ml_client.jobs.create_or_update(pipeline_job, experiment_name=experiment_name)
ml_client.jobs.stream(job.name)